### LLM in Databricks

![](/Workspace/Users/ghinku12345@gmail.com/calculate-embeddings-c6fa28b679c1cf21b1f93434a0ed927d.png)

In [0]:
%sql
drop table if exists question_data;

In [0]:
%sql
CREATE TABLE question_data (
    question_id INT,
    text STRING,
    category STRING
);

--training data
INSERT INTO question_data (question_id, text, category) VALUES
    (1,'What is the capital of France?', 'geography'),
    (2,'Name the tallest mountain in the world', 'geography'),
    (3,'How does quantum computing work?', 'technology'),
    (4,'Best practices for cybersecurity', 'technology'),
    (5,'Explain the theory of relativity', 'science'),
    (6,'What are the planets in our solar system?', 'science'),
    (7,'Who discovered electricity?', 'science'),
    (8,'What is the largest desert in the world?', 'geography'),
    (9,'Difference between classical and quantum computing?', 'technology'),
    (10,'What is the circumference of the Earth?', 'geography');


select * from question_data;

In [0]:
%sql
SELECT ai_query('databricks-bge-large-en',
    request => "Who discovered electricity?")

In [0]:
%sql
create or replace table question_data_vector_store as 
select question_id, TEXT,CATEGORY,ai_query('databricks-bge-large-en', request => text) as embedding_vector from question_data;

In [0]:
%sql
select * from question_data_vector_store;

In [0]:
%sql
ALTER TABLE `workspace`.`default`.`question_data_vector_store` SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
%sql

SELECT * FROM VECTOR_SEARCH(index => "workspace.default.first_vector_index", query_vector =>ai_query('databricks-bge-large-en', request => "Difference between Snowflake & Databricks?"), num_results => 2)

### Incremental Data Processing

In [0]:
%sql
INSERT INTO question_data (question_id, text, category) VALUES
    (11, 'What is photosynthesis?', 'science'),
    (12, 'Who invented the telephone?', 'technology'),
    (13, 'What is the capital of Australia?', 'geography'),
    (14, 'Define Newton’s laws of motion', 'science'),
    (15, 'What is artificial intelligence?', 'technology'),
    (16, 'Name the longest river in the world', 'geography'),
    (17, 'How does GPS work?', 'technology'),
    (18, 'What is the boiling point of water?', 'science'),
    (19, 'Where is the Sahara Desert located?', 'geography'),
    (20, 'What is blockchain technology?', 'technology');

In [0]:
%sql
select * from question_data;

In [0]:
%sql
select question_id, TEXT,  CATEGORY, ai_query('databricks-bge-large-en', request => text) as embedding_vector from question_data where question_id>(select max(question_id) from `workspace`.`default`.`question_data_vector_store`);

In [0]:
%sql
insert into question_data_vector_store (question_id, TEXT,  CATEGORY, embedding_vector)
select question_id, TEXT,  CATEGORY, ai_query('databricks-bge-large-en', request => text) as embedding_vector from question_data where question_id>(select max(question_id) from `workspace`.`default`.`question_data_vector_store`);

In [0]:
%sql
select * from question_data_vector_store;

In [0]:
%sql
SELECT * FROM VECTOR_SEARCH(index => "workspace.default.first_vector_index", query_vector =>ai_query('databricks-bge-large-en', request => "Explain God Particle, Higgs Boson"), num_results => 2)